In [1]:
import pandas as pd
import numpy as np

# --- 1. The Operational Data (Mocking your Field Data) ---
# Simulates the "Spirometry and Radiographic findings" mentioned in your letter
patients = pd.DataFrame({
    'patient_id': range(1001, 1021),
    'village_name': np.random.choice(['Kitoro', 'Lunyo', 'Nakiwogo', 'Lugonjo'], 20),
    'tb_treatment_outcome': np.random.choice(['Cured', 'Completed', 'Failed'], 20, p=[0.7, 0.2, 0.1]),
    'spirometry_fev1': np.random.normal(2.5, 0.5, 20), # Lung function metric
    'cxr_cavitation': np.random.choice([0, 1], 20) # 1 = Cavity present
})

# --- 2. The Geospatial Reference (Mocking the Village Coordinates) ---
# This represents the "Village-level coordinates" you mentioned
village_coords = pd.DataFrame({
    'village_name': ['Kitoro', 'Lunyo', 'Nakiwogo', 'Lugonjo'],
    'lat': [0.0600, 0.0500, 0.0400, 0.0700],
    'lon': [32.4700, 32.4600, 32.4500, 32.4800],
    'district_risk_zone': ['High', 'Medium', 'Low', 'High']
})

def run_surveillance_pipeline(patient_data, geo_data):
    """
    Integrates clinical data with geospatial registry to identify intervention targets.
    """
    print("--- Initiating Surveillance Pipeline ---")
    
    # Step 1: Linkage (The 'Spatial Registry' Concept)
    # Merging clinical outcomes with geographic location
    merged_registry = pd.merge(patient_data, geo_data, on='village_name', how='left')
    
    # Step 2: Risk Stratification Logic
    # Identifying patients who need "Targeted Screening" (Post-TB Complications)
    # Criteria: Treatment Failed OR (Low Lung Function AND High Risk Zone)
    
    conditions = [
        (merged_registry['tb_treatment_outcome'] == 'Failed'),
        (merged_registry['spirometry_fev1'] < 2.0) & (merged_registry['district_risk_zone'] == 'High')
    ]
    choices = ['Urgent Referral', 'Priority Screening']
    
    merged_registry['intervention_action'] = np.select(conditions, choices, default='Routine Follow-up')
    print(merged_registry)
    return merged_registry

# --- 3. Execution ---
final_registry = run_surveillance_pipeline(patients, village_coords)

# --- 4. Output for Decision Support ---
print("\n--- Targeted Intervention List (Generated via Python) ---")
high_priority = final_registry[final_registry['intervention_action'] != 'Routine Follow-up']
print(high_priority[['patient_id', 'village_name', 'intervention_action', 'lat', 'lon']])

# In a real deployment, this would export to a dashboard:
# final_registry.to_csv('modva_daily_surveillance.csv', index=False)

--- Initiating Surveillance Pipeline ---
    patient_id village_name tb_treatment_outcome  spirometry_fev1  \
0         1001      Lugonjo                Cured         1.978351   
1         1002      Lugonjo            Completed         2.897184   
2         1003      Lugonjo                Cured         3.210138   
3         1004      Lugonjo                Cured         2.609206   
4         1005        Lunyo                Cured         2.485007   
5         1006        Lunyo                Cured         2.930916   
6         1007     Nakiwogo            Completed         2.715876   
7         1008      Lugonjo                Cured         2.710007   
8         1009     Nakiwogo                Cured         2.597369   
9         1010       Kitoro            Completed         2.961873   
10        1011        Lunyo               Failed         2.187664   
11        1012     Nakiwogo               Failed         2.963502   
12        1013     Nakiwogo                Cured         2.333

In [8]:
final_registry

,patient_id,village_name,tb_treatment_outcome,spirometry_fev1,cxr_cavitation,lat,lon,district_risk_zone,intervention_action
0,1001,Lunyo,Failed,2.235382,0,0.05,32.46,Medium,Urgent Referral
1,1002,Kitoro,Completed,2.059207,1,0.06,32.47,High,Routine Follow-up
2,1003,Lunyo,Failed,2.750510,1,0.05,32.46,Medium,Urgent Referral
3,1004,Lugonjo,Cured,1.194543,1,0.07,32.48,High,Priority Screening
4,1005,Lunyo,Failed,2.213318,0,0.05,32.46,Medium,Urgent Referral
5,1006,Nakiwogo,Failed,2.579881,1,0.04,32.45,Low,Urgent Referral
6,1007,Kitoro,Cured,2.793631,0,0.06,32.47,High,Routine Follow-up
7,1008,Nakiwogo,Cured,2.563869,1,0.04,32.45,Low,Routine Follow-up
8,1009,Nakiwogo,Completed,1.557751,1,0.04,32.45,Low,Routine Follow-up
9,1010,Kitoro,Cured,2.445900,1,0.06,32.47,High,Routine Follow-up


In [6]:
village_coords

,village_name,lat,lon,district_risk_zone
0,Kitoro,0.06,32.47,High
1,Lunyo,0.05,32.46,Medium
2,Nakiwogo,0.04,32.45,Low
3,Lugonjo,0.07,32.48,High


In [7]:
final_registry


,patient_id,village_name,tb_treatment_outcome,spirometry_fev1,cxr_cavitation,lat,lon,district_risk_zone,intervention_action
0,1001,Lunyo,Failed,2.235382,0,0.05,32.46,Medium,Urgent Referral
1,1002,Kitoro,Completed,2.059207,1,0.06,32.47,High,Routine Follow-up
2,1003,Lunyo,Failed,2.750510,1,0.05,32.46,Medium,Urgent Referral
3,1004,Lugonjo,Cured,1.194543,1,0.07,32.48,High,Priority Screening
4,1005,Lunyo,Failed,2.213318,0,0.05,32.46,Medium,Urgent Referral
5,1006,Nakiwogo,Failed,2.579881,1,0.04,32.45,Low,Urgent Referral
6,1007,Kitoro,Cured,2.793631,0,0.06,32.47,High,Routine Follow-up
7,1008,Nakiwogo,Cured,2.563869,1,0.04,32.45,Low,Routine Follow-up
8,1009,Nakiwogo,Completed,1.557751,1,0.04,32.45,Low,Routine Follow-up
9,1010,Kitoro,Cured,2.445900,1,0.06,32.47,High,Routine Follow-up
